In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
#pdb.set_trace()
from ipywidgets import IntProgress
from IPython.display import display
import os
from selenium.webdriver.support import expected_conditions
from IPython.display import clear_output
from selenium.common.exceptions import WebDriverException
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from sc_header import createDriver,config_moneda

In [2]:
TIEMPO_DE_ESPERA = 5.5

start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

In [3]:
browser = createDriver()
browser.get('https://www.forever21.com/us/shop')
url_base = "https://www.forever21.com"

In [5]:
browser.save_screenshot('asd.png')

True

In [4]:
config_moneda(browser)

Cerrando modal


In [6]:
ipath = 'LINKS_F21.xlsx'
LINKS = pd.read_excel(ipath)

In [7]:
#LINKS = LINKS["PLUS_SIZE"].tolist()
LINKS = LINKS[LINKS['TIPO']=='PLUS + CURVE'].reset_index(drop=True)

In [11]:
#Si quieres hacer una lista c. Con elementos que no se encuentren en las dos listas, puedes generar por ejemplo, un bucle de 100 elementos:
#Y luego pasar el resultado a una lista c:
c = []
for i in range(100):
    if i not in a and i not in b:
        c.append(i)

#de manera pythonica quedaría asi:
c=[for i in range(100) if i not in a and i not in b]
        
#Y si lo que buscas es una lista con elementos de a que no esten en b, puedes hacer esto:
c=[]
for i in a:
    if i not in b:
        c.append(i)
#del mismo modo, podemos hacer una [list comprehension](https://recursospython.com/guias-y-manuales/comprension-de-listas-y-otras-colecciones/)
[i for i in a if i not in b]
#[i for i in a if i not in b]

0
6
10
13
14
18
19
20
21
23
24
26
27
28
31
32
33
34
35
36
37
38
39
40
42
43
44
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [7]:
lista_productos = []
ex_color = 0
for index,link in LINKS.iterrows():
    try:
        browser.get(link['HREF'])
    except WebDriverException:
        
        browser.quit()
        browser = createDriver()
        time.sleep(1)
        continue
        
    config_moneda(browser)
    tipologia = WebDriverWait(browser, 50).until(EC.presence_of_element_located((By.XPATH,'//*[@id="multi_submenu_title"]/h1'))).text
    
    
    max_page = max([int(i.text) for i in browser.find_elements_by_css_selector('#top-pager > ul > li') if i.text.isnumeric()])
               

    for page in range(max_page):

        #cantidad de productos dentro de la página.
        product_link = browser.find_elements_by_css_selector('a.item_slider.product_link')
        
        if browser.current_url == 'https://www.forever21.com/us/shop':
                break
        for indices, product in enumerate(product_link):
            
            try:
                URL = product.get_attribute('href')
            except:
                continue
        
            try:
                browser.execute_script(f"document.getElementsByClassName('qv square')[{indices}].click();")
            except:
                continue
            
            if browser.current_url == 'https://www.forever21.com/us/shop':
                break
            
            
            try:
                ELEMENT_COLORS = WebDriverWait(browser,30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,'#colorButton > li')))
                #ELEMENT_COLORS = browser.find_element_by_id('colorButton').find_elements_by_tag_name('li')
            except:
                time.sleep(3.5)
                continue

            for item, valor in enumerate(ELEMENT_COLORS):
                
                #browser.execute_script(f"document.getElementsByClassName('color')[{index_articulo}].getElementsByTagName('li')[{item}].getElementsByTagName('span')[0].getElementsByTagName('img')[0].click();")
                
                try:
                    if valor.is_displayed():
                        
                        valor.click()
                    #browser.execute_script(f"document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByClassName('color')[0].getElementsByTagName('li')[{item}].click();")
                except:
                    try:
                        browser.implicitly_wait(10)
                        browser.execute_script(f"document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByClassName('color')[0].getElementsByTagName('li')[{item}].click();")
                        ex_color += 1 
                    except:
                        break
                        
                
                
                NOMBRE_COLOR = valor.find_element_by_tag_name('img').get_attribute('alt')


                TALLES = browser.find_element_by_xpath('//*[@id="sizeButton"]').find_elements_by_tag_name('li')
                for TALLE in TALLES:

                    #TALLE BLOQUEADO ES OOS
                    if TALLE.find_element_by_tag_name('span').get_attribute('class') != "oos":
                        

                        NOMBRE_TALLE = TALLE.find_element_by_tag_name('span').text

                        # IMG PRODUCTO
                        
                        for _ in range(15):
                            try:
                                IMG = WebDriverWait(browser,50).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#modelImage_1_front_750 > img'))).get_attribute('src')
                                break
                            except:
                                time.sleep(2)
                            if IMG != 'https://www.forever21.com/us/shop/_common/images/c_loading.gif':
                                break
                        
                        # DESCRIPCION
                        DESCRIPCION = browser.find_element_by_class_name('p_name').text

                        # PRECIO
                        PRECIO = browser.find_element_by_css_selector('.p_price.pt_5.ws_100.row.mb_10').text
#                        browser.execute_script("var x = document.getElementsByClassName('p_price pt_5 ws_100 row mb_10')[0].textContent; return x").strip()
                        # CODIGO
                        try:
                            CODIGO = browser.find_element_by_css_selector('.t_mid.pt_10').text.split()[-1]
                        except:
                            
                            codigo = browser.execute_script("var x = document.getElementsByClassName('t_mid pt_10')[0].textContent; return x")
                            CODIGO = codigo.split()[-1]
                            

                        lista_productos.append([CODIGO,
                                                NOMBRE_TALLE,
                                                NOMBRE_COLOR,
                                                'PLUS SIZE',
                                                DESCRIPCION,
                                                PRECIO,
                                                IMG,
                                                URL,
                                                tipologia,
                                                page])

            
        #NEXTPAGE
        time.sleep(.8)
        try:
##            browser.execute_script("document.getElementsByClassName('p_next')[0].click();")
            #jquery click next page 
            browser.execute_script("$('#top-pager li span.p_next').click();")
        except:
            break
browser.quit()

In [17]:
df = pd.DataFrame(lista_productos)

In [18]:
len(df.drop_duplicates())

2841

In [ ]:
len(df)

In [20]:
df.rename(columns={0:'sku',
                   1:'talle',
                   2:'color',
                   3:'sexo',
                   4:'descripcion',
                   5:'precio',
                   6:'img',
                   7:'url',
                   8:'categoria'},inplace=True)
#df.drop_duplicates()

In [23]:
df.rename(columns={9:'drop'},inplace=True)

In [28]:
df[df['descripcion']=='Plus Size Leopard Print Cami Dress']

,sku,talle,color,sexo,descripcion,precio,img,url,categoria,9


In [24]:
df.drop('drop',axis=1,inplace=True)

In [25]:
df.drop_duplicates(inplace=True)

In [26]:
len(df)

3332

In [27]:
end_= datetime.datetime.now()

In [28]:
df.to_excel(f'./Salida/F21PLUS_SIZE{fecha}.xlsx')

In [ ]:
browser.quit()